In [67]:
import networkx as nx 
import numpy as np
import json
import pandas as pd
from sage.graphs.graph_input import from_dict_of_lists
from time import time
import sys

In [68]:
sys.setrecursionlimit(10000)

In [69]:
def nakljucni_MIS(G, P):
    I = []            #Nastajajoca neodvisna mnozica
    V = list(G.nodes)
    for i in V:        # V tej mnozici izberemo vozlisca, ki jih dodamo neodvisni mnozici. P je permutacija naravnih stevil do vkljucno len(V). Vozlisce dodamo I, ce ima najmanjso vrednost med sosedi.,tako tudi preprecimo ponavljanje vozlisc v mnozici.
        A = []      
        A.append(P[i])
        if list(G.neighbors(i)) != []:
            for j in list(G.neighbors(i)):
                A.append(P[j])
        if P[i] == min(A):
            I.append(i)
    N = [] 
    for i in I:
        N += G.neighbors(i)
    V1 = I + N
    V1 = list(dict.fromkeys(V1)) # V1 je mnozica, ki jo bomo odstranili iz grafa in je sestavljena iz vozlisc iz I, ter njihovih sosedov
    if I == []:
        return []
    else:
        G1 = G.copy()
        G1.remove_nodes_from(V1)
        return list(I + nakljucni_MIS(G1, P))

In [70]:
def tightness(graf, mnozica, v): #Dolocimo "tightness" vozlisca v, v mnozici X, tj. za vsakega soseda vozlisca v, v mnozici graf\X, tightness(v) povecamo za 1
    t = 0
    for w in graf.neighbors(v):
        if w in mnozica:
            t += 1
        else:
            pass
    return t

def lokalno_iskanje(G, I):
    I = list(I) # naredimo kopijo seznama I, oziroma ga pretvorimo, če ni v obliki seznama
    J = [] # seznam vozlišč, ki niso bila nadomeščena
    while I: # ponavljamo, dokler seznam ni prazen
        v = I.pop(0) # poberemo prvo vozlišče iz I
        L = [w for w in G.neighbors(v) if tightness(G, I+J, w) == 0]
        for i, v1 in enumerate(L):
            for w1 in L[i+1:]: # kopiramo iteratorja - da pregledamo vsak par enkrat
                if w1 not in G.neighbors(v1):
                    I += [v1, w1] # dodamo vozlišči v I
                    break
            else: # notranja zanka je prišla do konca
                continue # poskusimo z drugim v1
            break # notranjo zanko smo prekinili, prekinimo še zunanjo
        else: # zunanja zanka je prišla do konca
            J.append(v) # vozlišča v nismo uspeli nadomestiti, pustimo ga v izhodu
    return J

In [71]:
def CLP(G):
    p = MixedIntegerLinearProgram(maximization=True)
    x = p.new_variable(binary=True)
    p.set_objective(sum(x[v] for v in G))

    for u, v in G.edges(labels=False):
        p.add_constraint(x[u] + x[v] <= 1)
    
    p.solve()
    x = p.get_values(x)
    return [v for v, i in x.items() if i]

In [72]:
def zapisi_v_json(datoteka,A): 
    with open(datoteka +'.json','w') as js:
        js.write(
            '[' +
            ',\n'.join(json.dumps(i) for i in A) +
            ']\n')

# Dano verjetnost in n
def erdos_renyi(n,p,m):
    A = []
    for i in range(m):
        G = nx.erdos_renyi_graph(n,p)
        G_slovar = nx.to_dict_of_lists(G)
        A += [G_slovar]

    return A


# Razlicne n

def erdos_renyi_N(N, p):
    A = []
    for i in N:
        G = nx.erdos_renyi_graph(i,p)
        G_slovar = nx.to_dict_of_lists(G)
        A += [G_slovar]

    return A


#Razlicne p

def erdos_renyi_P(n, P):
    A = []
    for i in P:
        G = nx.erdos_renyi_graph(n,i)
        G_slovar = nx.to_dict_of_lists(G)
        A += [G_slovar]
    return A


# Ista vozlišča in verjetnost, večkrat


In [73]:
A = erdos_renyi(30, 0.3, 500)

In [74]:
np1000_nakljucno = []
np1000_najboljse = []
casi_np1000_nakljucno = []
casi_np1000_najboljsih = []
najboljse = []
resitve = []
for i in A:
    
    Q = []
    time1 = time()
    res = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
    time2 = time()
   

    time3 = time()
    nakljucne = []
    for x in range(20):
        nak = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
        Q.append(len(nak))
        nakljucne.append(nak)
    najboljsa_1 = max(Q)
    indeks = Q.index(najboljsa_1)
    maks_mn = nakljucne[indeks]
    time4 = time()

    np1000_nakljucno.append(len(res))
    casi_np1000_nakljucno.append(time2 - time1)
    casi_np1000_najboljsih.append(time4-time3)
    np1000_najboljse.append(najboljsa_1)
    najboljse.append(maks_mn)
    resitve.append(res)


In [75]:
np1000_CLP = []
casi_np1000_CLP = []
for i in A:
    X = Graph()
    from_dict_of_lists(X, i)
    time1 = time()
    rez = CLP(X)
    time2 = time()
    np1000_CLP.append(len(rez))
    casi_np1000_CLP.append(time2 - time1)

In [76]:
np1000_lokalno = []
casi_np1000_lokalno = []
for i in range(len(A)):
    time1 = time()
    rez = lokalno_iskanje(nx.Graph(A[i]), resitve[i])
    time2 = time()
    np1000_lokalno.append(len(rez))
    casi_np1000_lokalno.append(time2 - time1)


np1000_lokalno_najboljsi = []
casi_np1000_lokalno_najboljsi = []
for i in range(len(A)):
    time1 = time()
    rez = lokalno_iskanje(nx.Graph(A[i]), najboljse[i])
    time2 = time()
    np1000_lokalno_najboljsi.append(len(rez))
    casi_np1000_lokalno_najboljsi.append(time2 - time1)

np1000_lokalno_nakljucni = []
casi_np1000_lokalno_nakljucni = []
for i in A:
    nakljucne = []
    moci_mnozic = []

    time1 = time()
    for x in range(5):
        nak = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
        nakljucne.append(nak)
    for y in range(len(nakljucne)):
        rez = lokalno_iskanje(nx.Graph(i), nakljucne[y])
        moci_mnozic.append(len(rez))
    time2 = time()

    np1000_lokalno_nakljucni.append(max(moci_mnozic))
    casi_np1000_lokalno_nakljucni.append(time2-time1)
    

In [77]:
data_np1000 = {"Nakljucno":np1000_nakljucno, "Casi nakljucno": casi_np1000_nakljucno,
           "CLP": np1000_CLP, "Casi CLP": casi_np1000_CLP ,
            "Lokalno iskanje": np1000_lokalno, "Casi lokalno": casi_np1000_lokalno}
data_np1000 = pd.DataFrame(data_np1000)

data_np1000_maxi = {
    "Najboljsi nakljucno":np1000_najboljse, "Casi najboljsi nakljucno":casi_np1000_najboljsih,
    "CLP": np1000_CLP, "Casi CLP": casi_np1000_CLP ,
    "Najboljsi lokalno iskanje":np1000_lokalno_najboljsi, "Casi najboljsi lokalno iskanje":casi_np1000_lokalno_najboljsi, 
    "Nakljucni lokalno iskanje":np1000_lokalno_nakljucni, "Casi nakljucni lokalno iskanje":casi_np1000_lokalno_nakljucni, 
}

data_np1000_maxi = pd.DataFrame(data_np1000_maxi)

In [78]:

X = []
for i in range(len(np1000_lokalno)):
    X.append(np1000_CLP[i] - np1000_lokalno[i])

slovar_pojavitev = {}
for i in range(max(X)):
    slovar_pojavitev[i]=X.count(i)
kljuc = slovar_pojavitev.keys()
vrednost=slovar_pojavitev.values()

In [79]:
data_np1000_razlike = {
    "razlika":list(kljuc),
    "pojavitev":list(vrednost)
    }

data_np1000_razlike = pd.DataFrame(data_np1000_razlike)


# Različna vozlišča, ista verjetnost

In [80]:
N = list(range(1, 601, 8))
B = erdos_renyi_N(N, 0.005)

In [81]:
Np_nakljucno = []
casi_Np_nakljucno = []
resitve = []
for i in B:
    time1 = time()
    nak = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
    time2 = time()
    Np_nakljucno.append(len(nak))
    casi_Np_nakljucno.append(time2 - time1)
    resitve.append(nak)

In [82]:
Np_CLP = []
casi_Np_CLP = []
for i in B:
    X = Graph()
    from_dict_of_lists(X, i)
    time1 = time()
    rez = CLP(X)
    time2 = time()
    Np_CLP.append(len(rez))
    casi_Np_CLP.append(time2 - time1)

In [83]:
Np_lokalno = []
casi_Np_lokalno = []
for i in range(len(B)):
    time1 = time()
    rez = lokalno_iskanje(nx.Graph(B[i]), resitve[i])
    time2 = time()
    Np_lokalno.append(len(rez))
    casi_Np_lokalno.append(time2 - time1)

In [84]:
data_Np = {"Stevilo vozlisc":N ,"Nakljucno":Np_nakljucno, "Casi nakljucno": casi_Np_nakljucno,
           "CLP": Np_CLP, "Casi CLP": casi_Np_CLP ,
            "Lokalno iskanje": Np_lokalno, "Casi lokalno": casi_Np_lokalno}
data_Np = pd.DataFrame(data_Np)

# Ista vozlišča, različna verjetnost

In [85]:
P = list(np.linspace(0.1, 0.8, 80))
C = erdos_renyi_P(25, P)

In [86]:
nP_nakljucno = []
casi_nP_nakljucno = []
resitve = []
for i in C:
    time1 = time()
    nak = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
    time2 = time()
    nP_nakljucno.append(len(nak))
    casi_nP_nakljucno.append(time2 - time1)
    resitve.append(nak)

In [87]:
nP_CLP = []
casi_nP_CLP = []
for i in C:
    X = Graph()
    from_dict_of_lists(X, i)
    time1 = time()
    rez = CLP(X)
    time2 = time()
    nP_CLP.append(len(rez))
    casi_nP_CLP.append(time2 - time1)

In [88]:
nP_lokalno = []
casi_nP_lokalno = []
for i in range(len(C)):
    time1 = time()
    rez = lokalno_iskanje(nx.Graph(C[i]), resitve[i])
    time2 = time()
    nP_lokalno.append(len(rez))
    casi_nP_lokalno.append(time2 - time1)

In [89]:
data_nP = {"Verjetnost":P,"Nakljucno":nP_nakljucno, "Casi nakljucno": casi_nP_nakljucno,
           "CLP": nP_CLP, "Casi CLP": casi_nP_CLP ,
            "Lokalno iskanje": nP_lokalno, "Casi lokalno": casi_nP_lokalno}
data_nP = pd.DataFrame(data_nP)




In [90]:
zapisi_v_json("JSON_datoteke/grafi_ver",C)
data_nP.to_csv('R_koda/grafi_ver.csv')

In [91]:
zapisi_v_json("JSON_datoteke/grafi1000", A)
data_np1000.to_csv('R_koda/grafi1000.csv')

In [92]:
zapisi_v_json("JSON_datoteke/grafi_voz",B)
data_Np.to_csv('R_koda/grafi_voz.csv')

In [93]:
data_np1000_razlike.to_csv('R_koda/graf_razlik.csv')

In [94]:
data_np1000_maxi.to_csv('R_koda/grafi1000_maxi.csv')